# Incarceration & COVID-19

**How Jails Respond to COVID**<br>

This project scrapes daily jail roster information to create a large dataset. This dataset is designed to analyze how jail populations have fluctuated in response to COVID-19. Research centers on explaining why county jails in different parts of the United States have responded differently to the pandemic over time. 

A separate but related idea for this dataset analyzes the impact of pandemic-related jail population declines on local crime. This project uses daily jail roster population counts as the focal variable and the analysis uses group-based trajectory modeling. Our scraped data will address gaps in the [Vera](https://github.com/vera-institute/jail-population-data) dataset.

We start by comparing Washington and New York states because they dealt with COVID-19 at the early on-set of the pandemic. Below is a list of the data points to collect to harmonize with the Vera data.
- County Name
- State Name
- Daily Population Counts
- Reporting J
ail Name

## Imports

In [10]:
# Import Libraries
import re
import os
import time
import random
import requests
import numpy as np
import pandas as pd
from os import system   
from math import floor
from copy import deepcopy
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# import mysql.connector 
# from mysql.connector import errorcode
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Washington

In [8]:
pip install mysql

     |████████████████████████████████| 85 kB 2.2 MB/s eta 0:00:011
    ERROR: Command errored out with exit status 1:
     command: /opt/anaconda3/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/cy/wkp_87214x74hpqg5xwf6pbm0000gn/T/pip-install-6xdyul2b/mysqlclient/setup.py'"'"'; __file__='"'"'/private/var/folders/cy/wkp_87214x74hpqg5xwf6pbm0000gn/T/pip-install-6xdyul2b/mysqlclient/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/cy/wkp_87214x74hpqg5xwf6pbm0000gn/T/pip-install-6xdyul2b/mysqlclient/pip-egg-info
         cwd: /private/var/folders/cy/wkp_87214x74hpqg5xwf6pbm0000gn/T/pip-install-6xdyul2b/mysqlclient/
    Complete output (12 lines):
    /bin/sh: mysql_config: command not found
    /bin/sh: mariadb_config: command not found
    /bin/sh: mysql_config: command not found
   

In [ ]:
# Whitman
# http://www.whitmancountyjail.org/

# Spokane
# https://www.spokanecounty.org/352/Inmate-Roster

# Okanogan
# https://okanogansheriff.org/
# (under Daily Jail Inmate Log)

# Jefferson
# https://co.jefferson.wa.us/174/Jail-Inmate-Search
# (To view the full inmate roster click the Clear button then the Search button.)

# Grant
# https://www.grantcountywa.gov/SHERIFF/Corrections/Inmate-Roster.htm

# Gray's Harbor
# http://ghlea.com/JailRosters/GHCJRoster.html

# Ferry
# https://www.ferry-county.com/Courts%20and%20Law/Inmate%20Roster/Inmate_Roster_Page.html
# (in the section that says "MAY 11, 2020 - 8 inmates")

# Clallam
# https://websrv23.clallam.net/NewWorld.InmateInquiry/WA0050000/

# Adams
# https://www.co.adams.wa.us/government/jail_roster_and_booking_information/index.php
# (View Jail Roster Information)